In [11]:
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from bs4 import BeautifulSoup
import pandas as pd
import requests
from wiktionaryparser import WiktionaryParser
import re
import random

In [61]:
def convert_to_hot(syl_word):
    hot = []
    i = 0
    while  i < len(syl_word):
        if i == len(syl_word) - 1:
            hot += [1]
            return hot
        if syl_word[i+1] == '-':
            hot += [2]
            i += 2
        else:
            hot += [1]
            i += 1
    return hot

[1, 1, 2, 1, 1, 1]


In [75]:
print(orig_data[orig_data_syl.index('lo-re')])
print(orig_data_syl.index('flo-r-al'))
print(orig_data_eng.index('floral'))

Lo¥re
49217
49217


In [36]:
training_data_size = 10000
e2i_vocab_ortho = []
e2i_vocab_ipa = []

max_encoder_len = 0

x_tr_ortho = []
x_tr_ipa = []
y_tr = []

x_val_ortho = []
x_val_ipa = []
y_val = []

parser = WiktionaryParser()
orig_file = open("mhyph.txt", encoding='ISO-8859-1')

orig_data = orig_file.readlines()
orig_file.close()
orig_data = [line.strip('\n') for line in orig_data]
random.shuffle(orig_data)
orig_data_eng = [line.replace('¥', '').lower() for line in orig_data]
orig_data_syl = [line.replace('¥', '-').lower() for line in orig_data]

filtered_eng = []
filtered_syl = []
filtered_ipa = []

max_encoder_len_ortho = len(max(orig_data_eng, key=len))

for i in range(0, len(orig_data_eng)):
    if orig_data_eng[i].isalpha():
        try:
            ipa = parser.fetch(orig_data_eng[i])
        except:
            pass
        else:
            if ipa:
                if ipa[0]['pronunciations']['text']:
                    for e in ipa[0]['pronunciations']['text']:
                        if 'IPA: ' in e:
                            filtered_eng += [orig_data_eng[i]]
                            filtered_syl += [orig_data_syl[i]]
                            filtered_ipa += [e.split('IPA: ')[-1].split(',')[0]]
                            break
    print(str(i) + '/%i'%len(orig_data_eng), end='\r')

In [66]:
print(filtered_syl[filtered_syl.index('flo-r-al')])
print(filtered_syl.index('flo-r-al'))

flo-r-al
10018


In [56]:
x_tr_ortho = []
x_tr_ipa = []

y_tr = []

for line in filtered_eng:
    for c in line:
        if c not in e2i_vocab_ortho:
            e2i_vocab_ortho += [c]
            
for line in filtered_ipa:
    for c in line:
        if c not in e2i_vocab_ipa:
            e2i_vocab_ipa += [c]

e2i_ortho = dict((a,i) for i,a in enumerate(e2i_vocab_ortho, 1))
e2i_ipa = dict((a,i) for i,a in enumerate(e2i_vocab_ipa, 1))

for line in filtered_eng[:training_data_size]:
    converted = []
    for c in line:
        converted += [e2i_ortho[c]]
    x_tr_ortho += [converted]
    
x_tr_ortho = pad_sequences(x_tr_ortho, maxlen=max_encoder_len_ortho, padding='post')

for line in filtered_ipa[:training_data_size]:
    converted = []
    for c in line:
        converted += [e2i_ipa[c]]
    x_tr_ipa += [converted]
    
x_tr_ipa = pad_sequences(x_tr_ipa, maxlen=max_encoder_len_ipa, padding='post')

for line in filtered_syl[:training_data_size]:
    y_tr += [convert_to_hot(line)]

y_tr = pad_sequences(y_tr, maxlen=max_encoder_len_ipa, padding='post')



x_tr_file_ortho = open('data/x_tr_ortho.pkl','wb')
pickle.dump(x_tr_ortho, x_tr_file_ortho)
x_tr_file_ortho.close()

x_tr_file_ipa = open('data/x_tr_ipa.pkl','wb')
pickle.dump(x_tr_ipa, x_tr_file_ipa)
x_tr_file_ipa.close()

y_tr_file = open('data/y_tr.pkl','wb')
pickle.dump(y_tr, y_tr_file)
y_tr_file.close()

e2i_ortho_file = open('data/e2i_ortho.pkl','wb')
pickle.dump(e2i_ortho, e2i_ortho_file)
e2i_ortho_file.close()

e2i_ipa_file = open('data/e2i_ipa.pkl','wb')
pickle.dump(e2i_ipa, e2i_ipa_file)
e2i_ipa_file.close()

In [50]:
print(len(x_tr_ortho))
print(len(x_tr_ipa))
print(len(y_tr))

10000
10000
10000


In [57]:
x_val_ortho = []
x_val_ipa = []
y_val = []

for line in filtered_eng[training_data_size:]:
    converted = []
    for c in line:
        converted += [e2i_ortho[c]]
    x_val_ortho += [converted]
    
x_val_ortho = pad_sequences(x_val_ortho, maxlen=max_encoder_len_ortho, padding='post')

for line in filtered_ipa[training_data_size:]:
    converted = []
    for c in line:
        converted += [e2i_ipa[c]]
    x_val_ipa += [converted]
    
x_val_ipa = pad_sequences(x_val_ipa, maxlen=max_encoder_len_ipa, padding='post')

for line in filtered_syl[training_data_size:]:
    y_val += [convert_to_hot(line)]

y_val = pad_sequences(y_val, maxlen=max_encoder_len_ipa, padding='post')


x_val_file_ortho = open('data/x_val_ortho.pkl','wb')
pickle.dump(x_val_ortho, x_val_file_ortho)
x_val_file_ortho.close()

x_val_file_ipa = open('data/x_val_ipa.pkl','wb')
pickle.dump(x_val_ipa, x_val_file_ipa)
x_val_file_ipa.close()

y_val_file = open('data/y_val.pkl','wb')
pickle.dump(y_val, y_val_file)
y_val_file.close()

In [40]:
print(max_encoder_len_ortho)

45


In [52]:
print(len(x_val_ortho))
print(len(x_val_ipa))
print(len(y_val))

28132
28132
28132


In [59]:
max_encoder_len_ipa = len(max(filtered_ipa, key=len))
print(len(max(x_val_ipa, key=len)))

74
